Nhiệm vụ 1 – Đọc & làm sạch dữ liệu: 
Mục tiêu : hiểu cấu trúc dữ liệu thô và xử lý các vấn đề ảnh hưởng đến phân tích. 
- Chuẩn hóa patient_id và full_name.
- Xử lý age âm hoặc dạng text.
- Chuẩn hóa gender về M/F.
- Làm sạch visit_date, department, diagnosis.
- Làm sạch total_fee và insurance_coverage.
- Xuất các file sạch để dùng cho các nhiệm vụ sau


 Đường dẫn và đọc dữ liệu

In [2]:
import pandas as pd
import os

#  THIẾT LẬP ĐƯỜNG DẪN 

base_path = r"D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9"
input_path = os.path.join(base_path, "data_raw")
output_path = os.path.join(base_path, "data_clean")

print(f"Đọc dữ liệu từ: {input_path}")

#  ĐỌC DỮ LIỆU 
try:
    df_patient = pd.read_csv(os.path.join(input_path, "patient_info.csv"))
    df_visit = pd.read_csv(os.path.join(input_path, "visit_records.csv"))
    df_bill = pd.read_csv(os.path.join(input_path, "billing_info.csv"))
    print("-> Đã đọc thành công 3 file: Patient, Visit, Billing!")
except FileNotFoundError:
    print("LỖI: Không tìm thấy file. Hãy kiểm tra lại tên file hoặc đường dẫn.")
    raise

Đọc dữ liệu từ: D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_raw
-> Đã đọc thành công 3 file: Patient, Visit, Billing!


- Xử lý age âm hoặc dạng text.
- Chuẩn hóa gender về M/F.

In [3]:



# LÀM SẠCH PATIENT_INFO

print("\n[1/3] Đang xử lý: Patient Info...")

#  Chuẩn hóa ID và Tên
df_patient['patient_id'] = df_patient['patient_id'].astype(str).str.strip().str.upper()
df_patient['full_name'] = df_patient['full_name'].astype(str).str.strip().str.title()
df_patient['full_name'] = df_patient['full_name'].str.replace(r'\s+', ' ', regex=True)

#  Xử lý Tuổi
df_patient['age'] = df_patient['age'].astype(str).str.extract(r'(\d+)', expand=False)
df_patient['age'] = pd.to_numeric(df_patient['age'], errors='coerce').abs()

#  Chuẩn hóa Giới tính
gender_map = {'nam': 'M', 'm': 'M', 'male': 'M', 'nu': 'F', 'nữ': 'F', 'female': 'F', 'f': 'F'}
df_patient['gender'] = df_patient['gender'].astype(str).str.strip().str.lower().map(gender_map)
df_patient['gender'] = df_patient['gender'].fillna('Unknown')


# LÀM SẠCH VISIT_RECORDS

print("[2/3] Đang xử lý: Visit Records...")

#  Chuẩn hóa ID
df_visit['patient_id'] = df_visit['patient_id'].astype(str).str.strip().str.upper()
df_visit['visit_id'] = df_visit['visit_id'].astype(str).str.strip().str.upper()

#  Xử lý Ngày khám
df_visit['visit_date'] = pd.to_datetime(df_visit['visit_date'], dayfirst=True, errors='coerce')

#  Chuẩn hóa Khoa
df_visit['department'] = df_visit['department'].astype(str).str.strip().str.lower()
df_visit['department'] = df_visit['department'].str.replace(r'\s+', '', regex=True)
df_visit['department'] = df_visit['department'].str.title()

#  Xử lý Chẩn đoán
df_visit['diagnosis'] = df_visit['diagnosis'].astype(str).replace(r'[!@#$]+', float('nan'), regex=True)
df_visit['diagnosis'] = df_visit['diagnosis'].str.strip().str.lower()
df_visit['diagnosis'] = df_visit['diagnosis'].replace(['nan', ''], float('nan'))


#  LÀM SẠCH BILLING_INFO 

print("[3/3] Đang xử lý: Billing Info...")



# Chuẩn hóa ID (Để merge với Visit/Patient)
if 'patient_id' in df_bill.columns:
    df_bill['patient_id'] = df_bill['patient_id'].astype(str).str.strip().str.upper()
if 'visit_id' in df_bill.columns:
    df_bill['visit_id'] = df_bill['visit_id'].astype(str).str.strip().str.upper()

#  Xử lý Số tiền (Amount)

if 'amount' in df_bill.columns:
    #Chuyển sang string
    df_bill['amount'] = df_bill['amount'].astype(str)
    # Xóa dấu phẩy ngăn cách hàng nghìn 
    df_bill['amount'] = df_bill['amount'].str.replace(',', '')
    #  Xóa chữ 'VND', '$' hoặc khoảng trắng
    df_bill['amount'] = df_bill['amount'].str.replace(r'[^\d.]', '', regex=True)
    # : Chuyển sang số
    df_bill['amount'] = pd.to_numeric(df_bill['amount'], errors='coerce')
    # Điền 0 cho các giá trị lỗi (hoặc giữ NaN tùy bạn)
    df_bill['amount'] = df_bill['amount'].fillna(0)
#  Xử lý Trạng thái 
if 'status' in df_bill.columns:
    df_bill['status'] = df_bill['status'].astype(str).str.strip().str.title() 
#  Xử lý Ngày thanh toán
if 'payment_date' in df_bill.columns:
    df_bill['payment_date'] = pd.to_datetime(df_bill['payment_date'], dayfirst=True, errors='coerce')



[1/3] Đang xử lý: Patient Info...
[2/3] Đang xử lý: Visit Records...
[3/3] Đang xử lý: Billing Info...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_20160\2568362590.py:29: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_visit['visit_date'] = pd.to_datetime(df_visit['visit_date'], dayfirst=True, errors='coerce')


xuất file

In [4]:
# --- 4. LƯU DỮ LIỆU ---
if not os.path.exists(output_path):
    os.makedirs(output_path)

print(f"\nĐang lưu 3 file sạch vào: {output_path}")

df_patient.to_csv(os.path.join(output_path, "patient_clean.csv"), index=False, encoding='utf-8-sig')
df_visit.to_csv(os.path.join(output_path, "visit_clean.csv"), index=False, encoding='utf-8-sig')
df_bill.to_csv(os.path.join(output_path, "billing_clean.csv"), index=False, encoding='utf-8-sig')

print("-> HOÀN TẤT! Đã xử lý xong cả Patient, Visit và Billing.")


Đang lưu 3 file sạch vào: D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_clean
-> HOÀN TẤT! Đã xử lý xong cả Patient, Visit và Billing.
